# Randomized Incremental Construction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

In [ ]:
def generate_random_points(n_points=50, x_range=(0, 1), y_range=(0, 1), seed=None):
    """
    Generate uniformly distributed random points in 2D space.
    
    Args:
        n_points (int): Number of points to generate. Default is 50.
        x_range (tuple): Range for x-coordinates as (min, max). Default is (0, 1).
        y_range (tuple): Range for y-coordinates as (min, max). Default is (0, 1).
        seed (int, optional): Random seed for reproducibility. Default is None.
    
    Returns:
        np.ndarray: Array of shape (n_points, 2) containing the generated points.
    
    Example:
        >>> points = generate_random_points(n_points=100, seed=42)
        >>> points.shape
        (100, 2)
    """
    if seed is not None:
        np.random.seed(seed)

    x = np.random.uniform(x_range[0], x_range[1], n_points)
    y = np.random.uniform(y_range[0], y_range[1], n_points)
    
    points = np.column_stack((x, y))
    return points

def generate_gaussian_points(n_points=50, mean=(0.5, 0.5), cov=None, seed=None):
    """
    Generate 2D Gaussian distributed points.
    
    Creates points following a multivariate normal distribution. Useful for testing
    algorithms with clustered data patterns.
    
    Args:
        n_points (int): Number of points to generate. Default is 50.
        mean (tuple): Mean position (center) of the distribution. Default is (0.5, 0.5).
        cov (list or np.ndarray, optional): Covariance matrix (2x2). Default is isotropic 
            covariance [[0.02, 0.0], [0.0, 0.02]].
        seed (int, optional): Random seed for reproducibility. Default is None.
    
    Returns:
        np.ndarray: Array of shape (n_points, 2) containing points clipped to [0,1] range.
    
    Example:
        >>> points = generate_gaussian_points(n_points=100, mean=(0.3, 0.7), seed=42)
    """
    if seed is not None:
        np.random.seed(seed)
    if cov is None:
        cov = [[0.02, 0.0], [0.0, 0.02]]  # isotropic default covariance
    points = np.random.multivariate_normal(mean, cov, n_points)
    return np.clip(points, 0, 1)  # keep in [0,1] range


def generate_poisson_disc_points(radius=0.2, x_range=(0, 1), y_range=(0, 1), k=30, seed=None):
    """
    Generate points using Poisson disk sampling (Blue noise distribution).
    
    Creates a set of points where no two points are closer than the specified radius,
    resulting in a more uniform distribution than pure random sampling. This is useful
    for testing algorithms with well-spaced points.
    
    Args:
        radius (float): Minimum distance between any two points. Default is 0.2.
        x_range (tuple): Range for x-coordinates as (min, max). Default is (0, 1).
        y_range (tuple): Range for y-coordinates as (min, max). Default is (0, 1).
        k (int): Number of candidates to try per active point. Default is 30.
        seed (int, optional): Random seed for reproducibility. Default is None.
    
    Returns:
        np.ndarray: Array of shape (m, 2) where m varies based on radius and domain size.
    
    Algorithm:
        Uses Bridson's algorithm for fast Poisson disk sampling:
        1. Creates a grid for spatial lookup
        2. Starts with a random point
        3. Iteratively generates candidates around active points
        4. Accepts candidates that maintain minimum distance
    
    Example:
        >>> points = generate_poisson_disc_points(radius=0.05, seed=42)
    """
    if seed is not None:
        np.random.seed(seed)

    width, height = x_range[1] - x_range[0], y_range[1] - y_range[0]
    cell_size = radius / np.sqrt(2)
    grid_w, grid_h = int(np.ceil(width / cell_size)), int(np.ceil(height / cell_size))
    grid = -np.ones((grid_h, grid_w), dtype=int)
    points = []
    active_list = []

    def in_bounds(p):
        """Check if point is within the specified range."""
        return x_range[0] <= p[0] <= x_range[1] and y_range[0] <= p[1] <= y_range[1]

    def grid_coords(p):
        """Convert point coordinates to grid cell indices."""
        return int((p[0] - x_range[0]) / cell_size), int((p[1] - y_range[0]) / cell_size)

    def too_close(p):
        """Check if point is too close to any existing point."""
        gx, gy = grid_coords(p)
        for i in range(max(gy - 2, 0), min(gy + 3, grid_h)):
            for j in range(max(gx - 2, 0), min(gx + 3, grid_w)):
                idx = grid[i, j]
                if idx != -1:
                    dist = np.linalg.norm(points[idx] - p)
                    if dist < radius:
                        return True
        return False

    # Initial point
    first_point = np.array([
        np.random.uniform(x_range[0], x_range[1]),
        np.random.uniform(y_range[0], y_range[1])
    ])
    points.append(first_point)
    active_list.append(0)
    gx, gy = grid_coords(first_point)
    grid[gy, gx] = 0

    while active_list:
        idx = np.random.choice(active_list)
        base_point = points[idx]
        found = False
        for _ in range(k):
            theta = np.random.uniform(0, 2 * np.pi)
            r = np.random.uniform(radius, 2 * radius)
            new_point = base_point + np.array([r * np.cos(theta), r * np.sin(theta)])
            if in_bounds(new_point) and not too_close(new_point):
                points.append(new_point)
                gx, gy = grid_coords(new_point)
                grid[gy, gx] = len(points) - 1
                active_list.append(len(points) - 1)
                found = True
                break
        if not found:
            active_list.remove(idx)
    return np.array(points)


def generate_clustered_points(n_points=100, n_clusters=3, x_range=(0, 1), y_range=(0, 1), seed=None):
    """
    Generate multiple dense clusters with sparse regions between them.
    
    Creates a dataset with natural clustering patterns by generating multiple 
    Gaussian distributions at random locations. Useful for testing algorithm 
    performance on non-uniform data distributions.
    
    Args:
        n_points (int): Total number of points to generate. Default is 100.
        n_clusters (int): Number of distinct clusters. Default is 3.
        x_range (tuple): Range for x-coordinates as (min, max). Default is (0, 1).
        y_range (tuple): Range for y-coordinates as (min, max). Default is (0, 1).
        seed (int, optional): Random seed for reproducibility. Default is None.
    
    Returns:
        np.ndarray: Array of shape (n_points, 2) clipped to the specified range.
    
    Note:
        Points are distributed roughly equally among clusters, with any remainder
        distributed to the first clusters. Each cluster has a random covariance.
    
    Example:
        >>> points = generate_clustered_points(n_points=200, n_clusters=5, seed=42)
    """
    if seed is not None:
        np.random.seed(seed)
    cluster_centers = np.random.uniform([x_range[0], y_range[0]], [x_range[1], y_range[1]], (n_clusters, 2))
    points_per_cluster = n_points // n_clusters
    remainder = n_points % n_clusters
    points = []
    for i, center in enumerate(cluster_centers):
        size = points_per_cluster + (1 if i < remainder else 0)
        cov = np.diag(np.random.uniform(0.001, 0.01, 2))
        cluster = np.random.multivariate_normal(center, cov, size)
        points.append(cluster)
    return np.clip(np.vstack(points), 0, 1)

##### Implementation Code

In [ ]:
from __future__ import print_function
import math

def CalcDist(a, b):
    """
    Calculate Euclidean distance between two 2D points using Pythagorean theorem.
    
    Args:
        a, b (tuple): Points as (x, y) coordinates.
    
    Returns:
        float: Euclidean distance between points a and b.
    """
    return ((a[0] - b[0]) ** 2. + (a[1] - b[1]) ** 2.) ** 0.5

def CalcDistCached(pts, a, b, distCache):
    """
    Calculate distance with caching to avoid redundant computations.
    
    Args:
        pts (list): List of all points.
        a, b (int): Indices of the two points.
        distCache (dict): Cache dictionary mapping edge keys to distances.
    
    Returns:
        float: Cached or newly computed distance.
    """
    ptIds = (a, b)
    distId = min(ptIds), max(ptIds)
    if distId in distCache:
        return distCache[distId]
    
    dist = CalcDist(pts[a], pts[b])
    distCache[distId] = dist
    return dist

def RadialDistance(pts, seedIndex):
    """
    Sort all points by radial distance from a seed point.
    
    Args:
        pts (list): List of points as (x, y) tuples.
        seedIndex (int): Index of the seed point.
    
    Returns:
        list: Sorted list of (distance, point_index) tuples.
    """
    dists = []
    seedPt = pts[seedIndex]

    for ptNum, pt in enumerate(pts):
        dist = CalcDist(pt, seedPt)
        dists.append((dist, ptNum))

    dists.sort()
    return dists

def FindSmallestCircumCircle(pts, firstIndex, secondIndex):
    """
    Find the third point that creates the smallest circumcircle with two given points.
    
    Uses Heron's formula to compute the circumradius for each candidate triangle.
    
    Args:
        pts (list): List of all points.
        firstIndex, secondIndex (int): Indices of the two base points.
    
    Returns:
        list: Sorted list of (diameter, point_index) tuples.
    
    Raises:
        Exception: If the two base points are duplicates (zero distance).
    
    Mathematical Formula:
        For triangle with sides a, b, c:
        Area = sqrt(s(s-a)(s-b)(s-c)) where s = (a+b+c)/2
        Circumradius R = abc/(4*Area)
    """
    a = CalcDist(pts[firstIndex], pts[secondIndex])
    if a == 0.:
        raise Exception("Zero distance between duplicate points is not allowed")

    diams = []
    for ptNum, pt in enumerate(pts):
        if ptNum == firstIndex:
            continue
        if ptNum == secondIndex:
            continue
        b = CalcDist(pts[firstIndex], pts[ptNum])
        c = CalcDist(pts[secondIndex], pts[ptNum])

        # https://en.wikipedia.org/wiki/Heron%27s_formula#Numerical_stability
        x1 = (a+(b+c))
        x2 = (c-(a-b))
        x3 = (c+(a-b))
        x4 = (a+(b-c))
        x = x1*x2*x3*x4
        if x > 0.:
            sqrtx = x**0.5
            if sqrtx > 0.:
                diam = 0.5*a*b*c/sqrtx
                diams.append((diam, ptNum))
            else:
                # Prevent division by zero
                diams.append((float("inf"), ptNum))
        else:
            # Numerical instability detected
            diams.append((float("inf"), ptNum))
    
    diams.sort()
    return diams

def CircumCircleCentre(pta, ptb, ptc):
    """
    Calculate the center of the circumcircle for a triangle.
    
    Args:
        pta, ptb, ptc (tuple): Triangle vertices as (x, y) coordinates.
    
    Returns:
        tuple: (ux, uy) coordinates of the circumcircle center.
    
    Raises:
        RuntimeError: If triangle is degenerate (collinear points).
    
    Reference:
        https://en.wikipedia.org/wiki/Circumscribed_circle#Cartesian_coordinates
    """
    pta2 = (pta[0]**2.+pta[1]**2.)
    ptb2 = (ptb[0]**2.+ptb[1]**2.)
    ptc2 = (ptc[0]**2.+ptc[1]**2.)

    d = 2.*(pta[0]*(ptb[1]-ptc[1])+ptb[0]*(ptc[1]-pta[1])+ptc[0]*(pta[1]-ptb[1]))
    if d == 0.:
        raise RuntimeError("Could not find circumcircle centre")

    ux = (pta2*(ptb[1]-ptc[1]) + ptb2*(ptc[1]-pta[1]) + ptc2*(pta[1]-ptb[1])) / d
    uy = (pta2*(ptc[0]-ptb[0]) + ptb2*(pta[0]-ptc[0]) + ptc2*(ptb[0]-pta[0])) / d

    return ux, uy

def RightHandedCheck(pts, pt1, pt2, pt3):
    """
    Check if three points form a right-handed (counter-clockwise) orientation.
    
    Computes the z-component of the cross product of vectors (pt2->pt1) and (pt2->pt3).
    
    Args:
        pts (list): List of all points.
        pt1, pt2, pt3 (int): Indices of three points.
    
    Returns:
        float: Positive if CCW, negative if CW, zero if collinear.
    """
    vec21 = (pts[pt1][0] - pts[pt2][0], pts[pt1][1] - pts[pt2][1])
    vec23 = (pts[pt3][0] - pts[pt2][0], pts[pt3][1] - pts[pt2][1])
    return vec21[0] * vec23[1] - vec21[1] * vec23[0]

def FormTriangles(pts, seedTriangle, orderToAddPts):
    """
    Form triangles by incrementally adding points to a convex hull.
    
    Starting with a seed triangle, this function adds points one by one,
    updating the convex hull and creating new triangles for each visible edge.
    
    Args:
        pts (list): List of all points.
        seedTriangle (tuple): Initial triangle as (i, j, k) vertex indices.
        orderToAddPts (list): Ordered list of point indices to add.
    
    Returns:
        tuple: (hull, triangles) where:
            - hull: List of point indices forming the final convex hull
            - triangles: List of all triangles created as (i, j, k) tuples
    
    Algorithm:
        For each point to add:
        1. Determine which hull edges are visible from the point
        2. Remove visible edges from hull
        3. Create new triangles connecting point to visible edges
        4. Update hull to include the new point
    
    Raises:
        Exception: If no hull sides are visible or all sides are visible.
    """
    triangles = [seedTriangle]
    hull = seedTriangle[:]

    for ptToAdd in orderToAddPts:
        # Check which hull faces are visible
        visInd = []
        visList = []
        for hInd in range(len(hull)):
            vis = RightHandedCheck(pts, hull[hInd], hull[(hInd+1) % len(hull)], ptToAdd)
            visList.append(vis)
            if vis <= 0.:
                visInd.append(hInd)

        if len(visInd) == 0:
            raise Exception("No hull sides visible")

        # Check for range of sides that are visible
        firstSide = 0
        while firstSide in visInd:
            firstSide += 1
            if firstSide >= len(hull):
                raise Exception("No sides are not visible to point")

        while firstSide not in visInd:
            firstSide = (firstSide + 1) % len(hull)
        
        lastSide = firstSide
        while (lastSide+1) % len(hull) in visInd:
            lastSide = (lastSide+1) % len(hull)

        # Get copy of retained section of hull
        cursor = (lastSide + 1) % len(hull)
        newHull = []
        iterating = True
        while iterating:
            newHull.append(hull[cursor])
            if cursor in visInd:
                iterating = False
            cursor = (cursor + 1) % len(hull)

        # Add new point to hull
        newHull.append(ptToAdd)

        # Form new triangles
        cursor = firstSide
        iterating = True
        while iterating:
            tri = (hull[cursor], ptToAdd, hull[(cursor+1)%len(hull)])
            triangles.append(tri)

            if cursor == lastSide:
                iterating = False
            cursor = (cursor + 1) % len(hull)

        hull = newHull
    return hull, triangles

def CalcTriangleAng(pts, angleCache, pt1, pt2, pt3):
    """
    Calculate the angle at vertex pt3 in a triangle with caching.
    
    The angle is computed at pt3, with pt1 and pt2 defining the opposite side.
    
    Args:
        pts (list): List of all points.
        angleCache (dict): Cache for previously computed angles.
        pt1, pt2, pt3 (int): Point indices (angle computed at pt3).
    
    Returns:
        float: Angle in radians at vertex pt3.
    
    Raises:
        RuntimeError: If triangle has zero area (degenerate).
    
    Note:
        Handles both acute/obtuse angles and reflex angles correctly.
    """
    angId = (pt1, pt2, pt3)
    if angId in angleCache:
        return angleCache[angId]

    # Angle is computed on pt3. pt1 and pt2 define the side opposite the angle
    pt1v = pts[pt1]
    pt2v = pts[pt2]
    pt3v = pts[pt3]
    v31 = (pt1v[0] - pt3v[0], pt1v[1] - pt3v[1])
    v32 = (pt2v[0] - pt3v[0], pt2v[1] - pt3v[1])
    mv31 = (v31[0]**2. + v31[1]**2.) ** 0.5
    mv32 = (v32[0]**2. + v32[1]**2.) ** 0.5
    if mv31 == 0. or mv32 == 0.:
        raise RuntimeError("Angle not defined for zero area triangles")

    v31n = [c / mv31 for c in v31]
    v32n = [c / mv32 for c in v32]
    crossProd = - v31n[0] * v32n[1] + v31n[1] * v32n[0]
    dotProd = v31n[0] * v32n[0] + v31n[1] * v32n[1]
    
    # Limit to valid range
    if dotProd > 1.: dotProd = 1.
    if dotProd < -1.: dotProd = -1.

    if crossProd < 0.:
        # Reflex angle detected
        trigAng = 2. * math.pi - math.acos(dotProd)
    else:
        # Acute or obtuse angle
        trigAng = math.acos(dotProd)

    angleCache[angId] = trigAng
    return trigAng

def CheckAndFlipTrianglePair(pts, triOrdered1, triOrdered2, angleCache, distCache, debugMode = 0):
    """
    Check if two adjacent triangles need edge flipping for Delaunay property.
    
    Tests whether the sum of opposite angles exceeds π (Delaunay violation).
    If so, and if flipping improves the configuration, returns the flipped triangles.
    
    Args:
        pts (list): List of all points.
        triOrdered1, triOrdered2 (tuple): Adjacent triangles sharing an edge.
        angleCache (dict): Cache for angle computations.
        distCache (dict): Cache for distance computations.
        debugMode (int): If non-zero, perform additional validation checks.
    
    Returns:
        tuple: (flipNeeded, tri1, tri2) where:
            - flipNeeded (bool): True if flip was performed
            - tri1, tri2: Resulting triangles (flipped if needed, original otherwise)
    
    Algorithm (Delaunay Angle Test):
        For quadrilateral formed by two triangles:
        - If sum of opposite angles > π, edge is illegal
        - Flip creates edge connecting the other pair of vertices
        - Only flip if it improves the configuration
    
    Raises:
        RuntimeError: If debug mode detects invalid triangle orientation.
    """
    if debugMode and RightHandedCheck(pts, *triOrdered1) < 0.:
        raise RuntimeError("Left hand triangle detected", triOrdered1)
    if debugMode and RightHandedCheck(pts, *triOrdered2) < 0.:
        raise RuntimeError("Left hand triangle detected", triOrdered2)
    
    quad = triOrdered1[0], triOrdered1[2], triOrdered2[2], triOrdered2[1]

    try:
        t1 = CalcTriangleAng(pts, angleCache, quad[0], quad[2], quad[1])
        t3 = CalcTriangleAng(pts, angleCache, quad[2], quad[0], quad[3])
    except RuntimeError:
        return False, triOrdered1, triOrdered2

    flipDegenerateTri = (t1 == math.pi or t3 == math.pi)
    angTotal = t1 + t3
    flipForDelaunay = angTotal > math.pi

    if flipDegenerateTri or flipForDelaunay:
        try:
            t2 = CalcTriangleAng(pts, angleCache, quad[1], quad[3], quad[2])
            t4 = CalcTriangleAng(pts, angleCache, quad[3], quad[1], quad[0])
        except RuntimeError:
            return False, triOrdered1, triOrdered2

        if flipDegenerateTri and (t2 > math.pi or t4 > math.pi):
            # Flipping would create an overlap
            return False, triOrdered1, triOrdered2

        if t2 == math.pi or t4 == math.pi:
            # Flipping would create triangle of zero size
            return False, triOrdered1, triOrdered2

        flipTri1 = (triOrdered2[1], triOrdered1[2], triOrdered1[0])
        flipTri2 = (triOrdered1[2], triOrdered2[1], triOrdered1[1])
        flipAngTotal = t2 + t4
                
        if flipForDelaunay and flipAngTotal >= angTotal:
            # No improvement when flipped, so abort flip
            return False, triOrdered1, triOrdered2

        rhCheck1, rhCheck2 = 0., 0.
        if debugMode:
            rhCheck1 = RightHandedCheck(pts, *flipTri1)
            rhCheck2 = RightHandedCheck(pts, *flipTri2)

        # Ensure they are right handed
        if rhCheck1 < 0.:
            raise RuntimeError("Left hand triangle detected", flipTri1)
        if rhCheck2 < 0.:
            raise RuntimeError("Left hand triangle detected", flipTri2)

        return True, flipTri1, flipTri2

    return False, triOrdered1, triOrdered2

def HasCommonEdge(tri1, tri2):
    """
    Find the common edge between two triangles, if it exists.
    
    Args:
        tri1, tri2 (tuple): Triangles as (i, j, k) vertex tuples.
    
    Returns:
        tuple or None: ((edge_indices_in_tri1), (edge_indices_in_tri2)) or None.
    """
    edgeInd1 = [(0,1,2),(1,2,0),(2,0,1)]
    edgeInd2 = [(2,1,0),(1,0,2),(0,2,1)]
    for ei1 in edgeInd1:
        pt1 = tri1[ei1[0]]
        pt2 = tri1[ei1[1]]
        for ei2 in edgeInd2:
            if pt1 == tri2[ei2[0]] and pt2 == tri2[ei2[1]]:
                return (ei1, ei2)
    return None

def RemoveTriangleFromCommonEdges(sharedEdges, triangles, triNum):
    """
    Remove a triangle from the shared edges data structure.
    
    Args:
        sharedEdges (dict): Maps edge keys to lists of triangle indices.
        triangles (list): List of all triangles.
        triNum (int): Index of triangle to remove.
    """
    tri = triangles[triNum]
    edgeInds = [(0,1,2),(1,2,0),(2,0,1)]
    for edgeInd in edgeInds:
        edge = (tri[edgeInd[0]], tri[edgeInd[1]])
        edgeId = min(edge), max(edge)
        sharedEdges[edgeId].remove(triNum)    

def AddTriangleToCommonEdges(sharedEdges, triangles, triNum):
    """
    Add a triangle to the shared edges data structure.
    
    Args:
        sharedEdges (dict): Maps edge keys to lists of triangle indices.
        triangles (list): List of all triangles.
        triNum (int): Index of triangle to add.
    """
    tri = triangles[triNum]
    edgeInds = [(0,1,2),(1,2,0),(2,0,1)]
    for edgeInd in edgeInds:
        edge = (tri[edgeInd[0]], tri[edgeInd[1]])
        edgeId = min(edge), max(edge)
        if edgeId not in sharedEdges:
            sharedEdges[edgeId] = []
        sharedEdges[edgeId].append(triNum)

def FlipTriangles(pts, triangles, nodeOrdering = None):
    """
    Iteratively flip edges to enforce the Delaunay property.
    
    Processes all pairs of adjacent triangles, flipping edges that violate
    the Delaunay condition until no more flips are needed.
    
    Args:
        pts (list): List of all points.
        triangles (list): List of triangles (will be modified in-place).
        nodeOrdering (bool, optional): Enforce specific triangle winding order.
    
    Returns:
        list: Updated list of triangles satisfying Delaunay property.
    
    Raises:
        RuntimeError: If infinite loop detected (should not occur for valid input).
    
    Algorithm:
        1. Normalize all triangle orientations to CCW
        2. Build edge-to-triangle adjacency map
        3. For each edge with two adjacent triangles:
           - Check if Delaunay condition is violated
           - If yes, flip the edge
        4. Repeat until no flips occur
    """
    # Set all triangle windings the same way
    for triNum, tri in enumerate(triangles):
        rhCheck = RightHandedCheck(pts, *tri)
        if rhCheck < 0:
            triangles[triNum] = tri[::-1]
        if nodeOrdering == None and rhCheck != 0.:
            nodeOrdering = (rhCheck > 0.)

    # Catalog shared edges
    sharedEdges = {}
    for triNum, tri in enumerate(triangles):
        AddTriangleToCommonEdges(sharedEdges, triangles, triNum)

    angleCache = {}
    distCache = {}
    previousConfigurations = [triangles[:]]

    running = True
    while running:
        # Since we are modifying the edge structure, take a static copy of keys
        sharedEdgeKeys = list(sharedEdges.keys())

        count = 0

        for edgeKey in sharedEdgeKeys:
            edge = sharedEdges[edgeKey][:]
            if len(edge) < 2:
                continue

            tri1 = triangles[edge[0]]
            tri2 = triangles[edge[1]]

            commonEdge = HasCommonEdge(tri1, tri2)
            if commonEdge is None:
                raise Exception("Expected common edge")
            triInd1, triInd2 = commonEdge

            # Reorder nodes so the common edge is the first two vertices
            triOrdered1 = (tri1[triInd1[0]], tri1[triInd1[1]], tri1[triInd1[2]])
            triOrdered2 = (tri2[triInd2[0]], tri2[triInd2[2]], tri2[triInd2[1]])

            # Check if triangle flip is needed
            flipNeeded, ft1, ft2 = CheckAndFlipTrianglePair(pts, triOrdered1, triOrdered2, angleCache, distCache)
            
            if flipNeeded:
                RemoveTriangleFromCommonEdges(sharedEdges, triangles, edge[0])
                RemoveTriangleFromCommonEdges(sharedEdges, triangles, edge[1])

                triangles[edge[0]] = ft1
                triangles[edge[1]] = ft2

                AddTriangleToCommonEdges(sharedEdges, triangles, edge[0])
                AddTriangleToCommonEdges(sharedEdges, triangles, edge[1])

                count += 1

        if count > 0 and triangles in previousConfigurations:
            # Prevent an infinite loop of triangle flipping
            exception = RuntimeError("Cannot find delaunay arrangement")
            exception.triangles = triangles
            raise exception

        previousConfigurations.append(triangles[:])

        if count == 0:
            running = False

    if nodeOrdering is False:
        # Reverse order of triangles to match input node order        
        for triNum, tri in enumerate(triangles):
            triangles[triNum] = tri[::-1]        

    return triangles

def RemoveDuplicatePoints(pts):
    """
    Remove duplicate points from a list.
    
    Args:
        pts (list): List of points.
    
    Returns:
        list: List with duplicates removed.
    """
    filteredPts = set([tuple(pt) for pt in pts])
    return list(filteredPts)

def HeronsFormula(pts, tri):
    """
    Calculate triangle area using Heron's formula.
    
    Uses numerically stable version of Heron's formula.
    
    Args:
        pts (list): List of all points.
        tri (tuple): Triangle as (i, j, k) vertex indices.
    
    Returns:
        float: Area of the triangle (0 if degenerate).
    
    Reference:
        https://en.wikipedia.org/wiki/Heron%27s_formula#Numerical_stability
    """
    a = CalcDist(pts[tri[0]], pts[tri[1]])
    b = CalcDist(pts[tri[1]], pts[tri[2]])
    c = CalcDist(pts[tri[2]], pts[tri[0]])

    # https://en.wikipedia.org/wiki/Heron%27s_formula#Numerical_stability
    x1 = (a+(b+c))
    x2 = (c-(a-b))
    x3 = (c+(a-b))
    x4 = (a+(b-c))
    x = x1*x2*x3*x4
    if x < 0.:
        return 0.
    area = 0.25 * (x ** 0.5)
    return area

def RemoveZeroAreaTris(pts, triangles):
    """
    Filter out degenerate triangles with zero area or invalid angles.
    
    Args:
        pts (list): List of all points.
        triangles (list): List of triangles to filter.
    
    Returns:
        list: Filtered list containing only valid triangles.
    """
    filteredtris = []
    angleCache = {}

    for tri in triangles:
        area = HeronsFormula(pts, tri)

        ang1 = CalcTriangleAng(pts, angleCache, tri[2], tri[0], tri[1])
        ang2 = CalcTriangleAng(pts, angleCache, tri[0], tri[1], tri[2])
        ang3 = CalcTriangleAng(pts, angleCache, tri[1], tri[2], tri[0])
    
        if ang1 == 0. or ang2 == 0. or ang3 == 0.:
            continue

        if ang1 == math.pi or ang2 == math.pi or ang3 == math.pi:
            continue

        if area == 0.:
            continue

        filteredtris.append(tri)
    return filteredtris

def triangulate_sh(pts):
    """
    Compute Delaunay triangulation using the S-hull (Sweephull) algorithm.
    
    S-hull is a fast sweep-hull routine for Delaunay triangulation developed
    by David Sinclair. It combines radial sorting with incremental hull updates
    and edge flipping.
    
    Time Complexity: O(n log n) average case, O(n²) worst case
    Space Complexity: O(n)
    
    Args:
        pts (list): List of points as (x, y) tuples or lists.
    
    Returns:
        list: List of triangles as (i, j, k) tuples of vertex indices.
    
    Algorithm Overview:
        1. Select seed point (first point)
        2. Find nearest neighbor to seed
        3. Find third point creating smallest circumcircle (initial triangle)
        4. Sort remaining points by distance from initial circumcircle center
        5. Incrementally add points, updating convex hull
        6. Apply edge flipping to enforce Delaunay property
        7. Remove degenerate triangles
    
    Key Features:
        - Smart point ordering based on circumcircle distances
        - Incremental hull-based triangle formation
        - Post-processing edge flipping for Delaunay property
        - Robust handling of degenerate cases
    
    Example:
        >>> points = [(0, 0), (1, 0), (0.5, 1)]
        >>> triangles = triangulate_sh(points)
        >>> len(triangles)
        1
    
    References:
        S-hull: a fast sweep-hull routine for Delaunay triangulation
        by David Sinclair - http://www.s-hull.org/
    
    Raises:
        Exception: If invalid circumcircle or visibility errors occur.
    """
    # Select seed point
    seedIndex = 0

    # Sort by radial distance
    radialSorted = RadialDistance(pts, seedIndex)

    # Nearest point to seed point
    nearestToSeed = radialSorted[1][1]

    # Find third point that creates the smallest circum-circle
    sortedCircumCircles = FindSmallestCircumCircle(pts, seedIndex, nearestToSeed)
    if sortedCircumCircles[0][0] == float("inf"):
        raise Exception("Invalid circumcircle error")
    thirdPtIndex = sortedCircumCircles[0][1]

    # Order points to be right handed
    crossProd = RightHandedCheck(pts, seedIndex, nearestToSeed, thirdPtIndex)
    if crossProd < 0.:
        # Swap points
        secondPtInd = thirdPtIndex
        thirdPtIndex = nearestToSeed
    else:
        # Already right handed
        secondPtInd = nearestToSeed

    # Centre of circum-circle
    centre = CircumCircleCentre(pts[seedIndex], pts[secondPtInd], pts[thirdPtIndex])

    # Sort points by distance from circum-circle centre
    dists = []
    for ptNum, pt in enumerate(pts):
        if ptNum == seedIndex: continue
        if ptNum == secondPtInd: continue
        if ptNum == thirdPtIndex: continue
        
        dist = CalcDist(pts[ptNum], centre)
        dists.append((dist, ptNum))
    dists.sort()
    orderToAddPts = [v[1] for v in dists]

    # Form triangles by sequentially adding points
    hull, triangles = FormTriangles(pts, (seedIndex, secondPtInd, thirdPtIndex), orderToAddPts)

    # Flip adjacent pairs of triangles to meet Delaunay condition
    # https://en.wikipedia.org/wiki/Delaunay_triangulation#Visual_Delaunay_definition:_Flipping
    delaunayTris = FlipTriangles(pts, triangles)
    
    # Remove zero area triangles
    filteredTris = RemoveZeroAreaTris(pts, delaunayTris)

    return filteredTris


##### Comparison with SciPy Delaunay Triangulation

In [ ]:
# Verification Step
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import numpy as np

# 1. Generate Data
n_verify = 20
points_verify = generate_random_points(n_points=n_verify, seed=10)
np.random.shuffle(points_verify.tolist())

points_verify_gauss = generate_gaussian_points(n_points=n_verify, seed=42)
np.random.shuffle(points_verify_gauss)

points_verify_clustered = generate_clustered_points(n_points=n_verify, seed=52)
np.random.shuffle(points_verify_clustered)

r = np.sqrt(0.625 / n_verify) 
points_verify_poisson = generate_poisson_disc_points(radius=r, seed=62)
np.random.shuffle(points_verify_poisson)

# 2. Run Flip Algorithm
custom_triangles = triangulate_sh(points_verify.tolist())
custom_triangles_gauss = triangulate_sh(points_verify_gauss.tolist())
custom_triangles_clustered = triangulate_sh(points_verify_clustered.tolist())
custom_triangles_poisson = triangulate_sh(points_verify_poisson.tolist())
# 3. Run Scipy Delaunay
delaunay = Delaunay(points_verify)
delaunay_gauss = Delaunay(points_verify_gauss)
delaunay_clustered = Delaunay(points_verify_clustered)
delaunay_poisson = Delaunay(points_verify_poisson)

scipy_simplices = delaunay.simplices
scipy_simplices_gauss = delaunay_gauss.simplices
scipy_simplices_clustered = delaunay_clustered.simplices
scipy_simplices_poisson = delaunay_poisson.simplices

# 4. Compare Results for all datasets
datasets = [
    ("Random", custom_triangles, scipy_simplices, points_verify),
    ("Gaussian", custom_triangles_gauss, scipy_simplices_gauss, points_verify_gauss),
    ("Clustered", custom_triangles_clustered, scipy_simplices_clustered, points_verify_clustered),
    ("Poisson", custom_triangles_poisson, scipy_simplices_poisson, points_verify_poisson)
]

print(f"Verification with {n_verify} points for each dataset:\n")
for name, custom, scipy, _ in datasets:
    custom_set = set(tuple(sorted(t)) for t in custom)
    scipy_set = set(tuple(sorted(t)) for t in scipy)
    is_correct = (custom_set == scipy_set)
    
    print(f"{name:12} - RIC: {len(custom_set):3d} triangles, Scipy: {len(scipy_set):3d} triangles, Match: {is_correct}")
    
    if not is_correct:
        print(f"  → In RIC only: {custom_set - scipy_set}")
        print(f"  → In Scipy only: {scipy_set - custom_set}")

# 5. Visualize All Datasets in a Grid
fig, axes = plt.subplots(4, 2, figsize=(12, 16))

for idx, (name, custom, scipy, points) in enumerate(datasets):
    # Scipy Plot (left column)
    axes[idx, 0].set_title(f"{name} - Scipy Delaunay")
    if len(scipy) > 0:
        axes[idx, 0].triplot(points[:, 0], points[:, 1], scipy, color='green')
    axes[idx, 0].plot(points[:, 0], points[:, 1], 'ko', markersize=4)
    axes[idx, 0].set_aspect('equal')
    axes[idx, 0].grid(True, alpha=0.3)
    
    # RIC Plot (right column)
    axes[idx, 1].set_title(f"{name} - Sweephull")
    if custom:
        axes[idx, 1].triplot(points[:, 0], points[:, 1], [list(t) for t in custom], color='blue')
    axes[idx, 1].plot(points[:, 0], points[:, 1], 'ko', markersize=4)
    axes[idx, 1].set_aspect('equal')
    axes[idx, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Dataset Testing

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

ns = [i for i in range(1000, 10000, 500)]

results = {
    "Random": [],
    "Gaussian": [],
    "Clustered": [],
    "Poisson": []
}

for n in ns:
    print(f"Running for n={n}...")
    
    # Random
    pts = generate_random_points(n_points=n, seed=10)
    np.random.shuffle(pts)
    start_time = time.time()
    triangulate_sh(pts.tolist())
    end_time = time.time()
    results["Random"].append(end_time - start_time)
    
    # Gaussian
    pts = generate_gaussian_points(n_points=n, seed=42)
    np.random.shuffle(pts) # Shuffle to remove input order bias
    start_time = time.time()
    triangulate_sh(pts.tolist())
    end_time = time.time()
    results["Gaussian"].append(end_time - start_time)
    
    # Clustered
    pts = generate_clustered_points(n_points=n, seed=52)
    np.random.shuffle(pts) # Shuffle to remove input order bias
    start_time = time.time()
    triangulate_sh(pts.tolist())
    end_time = time.time()
    results["Clustered"].append(end_time - start_time)
    
    # Poisson
    r = np.sqrt(0.625 / n) 
    pts = generate_poisson_disc_points(radius=r, seed=62)
    np.random.shuffle(pts) # Shuffle to remove input order bias
    start_time = time.time()
    triangulate_sh(pts.tolist())
    end_time = time.time()
    results["Poisson"].append((len(pts), end_time - start_time))

# Plotting
plt.figure(figsize=(12, 8))
plt.plot(ns, results["Random"], label="Random (Uniform)", marker='o')
plt.plot(ns, results["Gaussian"], label="Gaussian", marker='s')
plt.plot(ns, results["Clustered"], label="Clustered", marker='^')

# Poisson scatter/line
p_ns = [x[0] for x in results["Poisson"]]
p_times = [x[1] for x in results["Poisson"]]

# Sort by n (just in case)
p_sorted = sorted(zip(p_ns, p_times))
p_ns = [x[0] for x in p_sorted]
p_times = [x[1] for x in p_sorted]

plt.plot(p_ns, p_times, label="Poisson Disc", marker='x')

# Theoretical Lines
ns_arr = np.array(ns)

# Use the maximum time from Random as a reference for scaling
# We use the last point to anchor the theoretical curves
if results["Random"]:
    # 1. Fit O(n log n) to the LAST point (Max N)
    # This checks: "Does my data follow this shape?"
    ref_n_last = ns[-1]
    ref_time_last = results["Random"][-1]
    
    k_log = ref_time_last / (ref_n_last * np.log(ref_n_last))
    plt.plot(ns, k_log * (ns_arr * np.log(ns_arr)), 'r--', label="O(n log n) (Fitted to End)", linewidth=2)

    # 2. Fit O(n^2) to an EARLY point (e.g., the 5th point)
    # This checks: "If this were quadratic, where would it be by now?"
    # We skip index 0-4 to avoid initial startup noise/caching jitter
    ref_idx = 5 
    if len(ns) > ref_idx:
        ref_n_early = ns[ref_idx]
        ref_time_early = results["Random"][ref_idx]
        k2 = ref_time_early / (ref_n_early**2)
        plt.plot(ns, k2 * (ns_arr**2), 'k--', label="O(n^2) (Fitted to Start)", linewidth=2)

plt.xlabel("Number of Points (n)")
plt.ylabel("Time Taken (seconds)")
plt.title("Time Taken vs Number of Points (Shuffled Input)")
plt.legend()
plt.grid(True)
plt.show()